In [1]:
"""from datasets import load_dataset
import pandas as pd

scicite = load_dataset('sst2')

# Convert to a pandas DataFrame
df = pd.DataFrame({
    'text': scicite['train']['sentence'],
    'label': scicite['train']['label'],
})

# Save to CSV
df.to_csv('sst2.csv', index=False)"""

"from datasets import load_dataset\nimport pandas as pd\n\nscicite = load_dataset('sst2')\n\n# Convert to a pandas DataFrame\ndf = pd.DataFrame({\n    'text': scicite['train']['sentence'],\n    'label': scicite['train']['label'],\n})\n\n# Save to CSV\ndf.to_csv('sst2.csv', index=False)"

In [2]:
import pandas as pd

df = pd.read_csv('../datasets/sst2.csv')
df.head()

,text,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0


In [3]:
df.shape

(67349, 2)

In [4]:
"""
Now we do Data Preprocessing.
"""
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

regexp = RegexpTokenizer("[\w']+")

def convert_to_lowercase(text):
    return text.lower()
def remove_whitespace(text):
    return text.strip()
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "") # discarding apostrophe from the string to keep the contractions intact
    return text.translate(str.maketrans("", "", punct_str))
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_http(text):
    http = "https?://\S+|www\.\S+" # matching strings beginning with http (but not just "http")
    pattern = r"({})".format(http) # creating pattern
    return re.sub(pattern, "", text)
# Stopwords
stops = stopwords.words("english") # stopwords
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"] # additional stopwords
allstops = stops + addstops
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in allstops])
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

In [5]:
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text) # converting text to one line
    text = re.sub('\[.*?\]', '', text) # removing square brackets
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    return text

In [6]:
df['text'] = df['text'].apply(text_normalizer)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,2), max_features=60000, smooth_idf=True, sublinear_tf=True)
df_feature = vectorizer.fit_transform(df['text'])
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_feature, df['label'], test_size=0.2, random_state=42)

In [18]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

dt_model = DecisionTreeClassifier()
dt_model.fit(df_features_train, df_target_train)
target_prediction = dt_model.predict(df_features_test)

In [19]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df_target_test, target_prediction))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5909
           1       0.91      0.92      0.91      7561

    accuracy                           0.90     13470
   macro avg       0.90      0.90      0.90     13470
weighted avg       0.90      0.90      0.90     13470

